##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Tracking metrics with TensorBoard
<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/migrate/tensorboard">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/migrate/tensorboard.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/migrate/tensorboard.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/migrate/tensorboard.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>


## Migrating Tensorboard metric displays from TF1 to TF2

`TensorBoard` is a built-in visualization tool for graphing metrics in TF. Common metrics to be visualized in `TensorBoard` include accuracy and loss. `TensorBoard` is compatible with TF1 and TF2 code. In TF1, `tf.estimator.Estimator` saves summaries for `TensorBoard` by default, and in TF2, summaries can be saved using `tf.keras.callbacks.TensorBoard`. In this example, you will see code that visualizes training progress using `TensorBoard` in TF1, followed by its equivalency in TF2.

In [ ]:
import tensorflow.compat.v1 as tf1
import tensorflow as tf
import numpy as np
import datetime
%load_ext tensorboard

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

### TF1: MNIST Estimator training with visualization

In this example, you will see a premade `tf.estimator.DNNClassifier` trained and evaluated on MNIST data and visualized using `Tensorboard`.

In [ ]:
%reload_ext tensorboard

feature_columns = [tf1.feature_column.numeric_column("x", shape=[28, 28])]

config = tf1.estimator.RunConfig(save_summary_steps=1,
                                 save_checkpoints_steps=1)

path = tempfile.mkdtemp()

classifier = tf1.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[256, 32],
    optimizer=tf1.train.AdamOptimizer(0.001),
    n_classes=10,
    dropout=0.1,
    model_dir=path,
    config = config
)

train_input_fn = tf1.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=y_train.astype(np.int32),
    num_epochs=10,
    batch_size=50,
    shuffle=True,
)

test_input_fn = tf1.estimator.inputs.numpy_input_fn(
    x={"x": x_test},
    y=y_test.astype(np.int32),
    num_epochs=10,
    shuffle=False
)

train_spec = tf1.estimator.TrainSpec(input_fn=train_input_fn, max_steps=10)
eval_spec = tf1.estimator.EvalSpec(input_fn=test_input_fn,
                                   steps=10,
                                   throttle_secs=0)

tf1.estimator.train_and_evaluate(estimator=classifier,
                                train_spec=train_spec,
                                eval_spec=eval_spec)

In [ ]:
%tensorboard --logdir {classifier.model_dir}

<!-- <img class="tfo-display-only-on-site" src="images/tensorboard_TF1.png"/> -->

### TF2: Model training with visualization

You will see a multi-layered `tf.keras.Model` trained with visualization using a `tf.keras.callbacks.TensorBoard`. In the TensorBoard graph you see the accuracy and loss by epoch when `Model.fit()` is called, which includes training and validation runs.

In [ ]:
%reload_ext tensorboard

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],
              steps_per_execution=10)

log_dir = tempfile.mkdtemp()
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,
                                                      histogram_freq=1)

model.fit(x=x_train,
          y=y_train,
          epochs=10,
          validation_data=(x_test, y_test),
          callbacks=[tensorboard_callback])

In [ ]:
%tensorboard --logdir {tensorboard_callback.log_dir}

<!-- <img class="tfo-display-only-on-site" src="images/tensorboard_TF2.png"/> -->


# Next Steps


*   You can read more about `TensorBoard` at https://www.tensorflow.org/tensorboard/get_started 
*   If you're interested in lower level APIs see the `tf.summary` migration guide https://www.tensorflow.org/tensorboard/migrate
